# What is new in Qadence 2?



## Syntax



<div align="center">
<figure>
<img src="fig1.png" width="500"/>
<figcaption>From Qadence tutorial</figcaption>
<figure>
</div>

Mathematically
$$
X_0 \otimes Y_1 \otimes RX_2(x) \otimes RX_3(0.5) \otimes CX_{2,3} \otimes e^{-it\sum_{i=0}^3 Z_i}
$$

or

$$
X_0 Y_1 \, RX_2(x) \, RX_3(0.5) \, CX_{2,3} \, e^{-it\sum_{i=0}^3 Z_i}
$$


In [1]:
from qadence2.expressions import *

x = parameter("x")
t = parameter("t")

X(0) * Y(1) * RX(x)(2) * RX(0.5)(3) * X(target=(3,), control=(2,)) * exp(-1j * t * sum(Z(i) for i in range(4)))

X[0] Y[1] RX(x)[2] RX(0.5)[3] X[3| 2] E ^ ((-0-1j) t Z[0] + (-0-1j) t Z[1] + (-0-1j) t Z[2] + (-0-1j) t Z[3])[0, 1, 2, 3]

In [2]:
RX(x)(2) * RX(0.5)(3) * X(target=(3,), control=(2,)) * X(0) * Y(1) * exp(-1j * t * sum(Z(i) for i in range(4)))

X[0] Y[1] RX(x)[2] RX(0.5)[3] X[3| 2] E ^ ((-0-1j) t Z[0] + (-0-1j) t Z[1] + (-0-1j) t Z[2] + (-0-1j) t Z[3])[0, 1, 2, 3]

### Different types of symbols for different purposes



Qadence 2 is designed for QML and supports specialised types for different optimisation strategies. For instance, `parameter` types are treated as constants and only `variable` types are considered during gradient calculations.

In [3]:
a = parameter("a")
b = parameter("b")
phi = variable("phi")

### Automatic basic simplifications



Distributive property.

In [4]:
a * (cos(phi / 2) * X(1) - sin(phi / 2) * Y(1)) - b * Z(1)

a cos(0.5 phi) X[1] - a sin(0.5 phi) Y[1] - b Z[1]

Unitary Hermitian operatins automatic cancelation.

In [5]:
X(3) * (Y(1) + Z(2)) * Z(2)

Y[1] Z[2] X[3] + X[3]

### Subspace



Local operations can be spanned over multiple qubits; `X(1, 2)` is mathematically equivalent to $X_1 \otimes X_2$.

Any operation can be used as a controlled operation by providing `target` and `control` indices.

In [6]:
X(target=(1,), control=(0,2))

X[1| 0, 2]

### Declaring new operators



Direct declaration.

In [7]:
MyGate = unitary_hermitian_operator('MyGate')

MyGate(0, 1) * MyGate(0, 1)

1.0

As a regular Python function.

In [8]:
n = lambda l: (1 - Z(l)) / 2

n(1) * n(2) + n(2) * n(3)

0.5 - 0.5 Z[2] - 0.25 Z[1] + 0.25 Z[1] Z[2] - 0.25 Z[3] + 0.25 Z[2] Z[3]

### Hardware elements without implementation details



In this example, the `FreeEvolution(2.5)(1, 2)` indicates an interaction between qubits 1 and 2 over 2.5 unit times. The backend decides how the unit time is defined and how the dynamic decoupling is performed.

In [9]:
t = time_variable("t")
omega = array_variable("omega", 4)
detuning = array_variable("detuning", 3)
phase = parameter("phase")

expr = (
    NativeDrive(t / 2, omega, detuning, phase)()
    * FreeEvolution(2.5)(1, 2)
    * NativeDrive(t / 2, omega, -detuning, phase)()
)

expr

NativeDrive(0.5 t, omega, detuning, phase)[*] FreeEvolution(2.5)[1, 2] NativeDrive(0.5 t, omega, -detuning, phase)[*]

## 🚧 Symbolic analysis

In [10]:
x = parameter("x")
theta = variable("theta")
expr = RX(x/2)() * CZ(0, 1) * RX(theta/2)(0)
expr

RX(0.5 x)[*] CZ[0, 1] RX(0.5 theta)[0]

In [11]:
replace(expr, {CZ(0, 1): FreeEvolution(2.7)()})

RX(0.5 x)[*] FreeEvolution(2.7)[*] RX(0.5 theta)[0]

## Intermidiate representation

In [12]:
expr = RX(a * phi / 2)(2) * CZ() * RY(b * phi / 2)(0)

irc(expr)

Model(
  register=AllocQubits(
    num_qubits=3,
    qubit_positions=[],
    grid_type=None,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    'a': Alloc(1, trainable=False),
    'phi': Alloc(1, trainable=True),
    'b': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('a'))),
    Assign('%1', Call('mul', Load('%0'), Load('phi'))),
    QuInstruct('rx', Support(target=(2,)), Load('%1')),
    QuInstruct('cz', Support.target_all()),
    Assign('%2', Call('mul', 0.5, Load('b'))),
    Assign('%3', Call('mul', Load('%2'), Load('phi'))),
    QuInstruct('ry', Support(target=(0,)), Load('%3')),
  ],
  directives={},
  settings={},
)

In [13]:
reset_ir_options()
set_qubits_positions([(-1,0), (-1, 1), (1, 0), (1, 1)])
set_grid_type("triangular")

t = time_variable("t")
omega = array_variable("omega", 4)
detuning = array_variable("detuning", 3)
phase = parameter("phase")

expr = (
    NativeDrive(t / 2, omega, detuning, phase)()
    * FreeEvolution(2.5)(1, 2)
    * NativeDrive(t / 2, omega, -detuning, phase)()
)

irc(expr)

Model(
  register=AllocQubits(
    num_qubits=4,
    qubit_positions=[(-1, 0), (-1, 1), (1, 0), (1, 1)],
    grid_type=triangular,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    't': Alloc(1, trainable=True, attrs={'time_parameter': True}),
    'omega': Alloc(4, trainable=True),
    'detuning': Alloc(3, trainable=True),
    'phase': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('t'))),
    QuInstruct('dyn_pulse', Support.target_all(), Load('%0'), Load('omega'), Load('detuning'), Load('phase')),
    QuInstruct('dyn_wait', Support(target=(1, 2)), 2.5),
    Assign('%1', Call('mul', -1.0, Load('detuning'))),
    QuInstruct('dyn_pulse', Support.target_all(), Load('%0'), Load('omega'), Load('%1'), Load('phase')),
  ],
  directives={},
  settings={},
)

## Pulser example

In [14]:
from qadence2.platforms.compiler import compile
from qadence2.platforms.types import DeviceName

reset_ir_options()
set_qubits_positions([(0,0), (0, 1), (1, 0)])
set_grid_type("triangular")
add_qpu_directives({"enable_digital_analog": True})

x = parameter("x")
expr = RX(x/2)()

model = irc(expr)
api = compile(model, "pulser", DeviceName.FRESNEL_EOM)
result = api.run(num_shots=1000, values={"x": 3.14})

print(expr, model, sep="\n-----\n")
result

RX(0.5 x)[*]
-----
Model(
  register=AllocQubits(
    num_qubits=3,
    qubit_positions=[(0, 0), (0, 1), (1, 0)],
    grid_type=triangular,
    grid_scale=1.0,
    options={},
  ),
  inputs={
    'x': Alloc(1, trainable=False),
  },
  instructions=[
    Assign('%0', Call('mul', 0.5, Load('x'))),
    QuInstruct('rx', Support.target_all(), Load('%0')),
  ],
  directives={
    'enable_digital_analog': True,
  },
  settings={},
)


Counter({'101': 137,
         '011': 133,
         '000': 130,
         '010': 125,
         '001': 124,
         '100': 119,
         '110': 117,
         '111': 115})